In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

# Imports
import numpy as np
import tensorflow as tf

In [2]:
tf.logging.set_verbosity(tf.logging.INFO)

In [3]:
# model modes TRAIN, EVAL, PREDICT
def cnn_model_fn(features, labels, mode):
    '''Model fuction for CNN'''
    # Input Layer
    batch_size = -1
    image_width = 28
    image_height = 28
    # channel 1 for greyscale, 3 for rgb
    channels = 1
    input_layer = tf.reshape(
        features['x'],
        [batch_size, image_width,image_height, channels])

    # Convolutional Layer #1
    '''
    Applies 32 5x5 filters(extracting 5x5-pixel subregions),
    with ReLU activation function
    '''
    conv1 = tf.layers.conv2d(
        inputs=input_layer,
        filters=32,
        kernel_size=[5, 5],
        padding='same',
        activation=tf.nn.relu)

    # Pooling Layer #1
    '''
    Performs max pooling with a 2x2 filter and stride of 2
    (which specifies that pooled regions do not overlap)
    '''
    pool1 = tf.layers.max_pooling2d(
        inputs=conv1,
        pool_size=[2, 2],
        strides=2)

    # Convolutional Layer #2
    '''
    Applies 64 5x5 filters, with ReLU activation function
    '''
    conv2 = tf.layers.conv2d(
        inputs=pool1,
        filters=64,
        kernel_size=[5, 5],
        padding='same',
        activation=tf.nn.relu)

    # Pooling Layer #2
    '''
    Performs max pooling with a 2x2 filter and stride of 2
    '''
    pool2 = tf.layers.max_pooling2d(
        inputs=conv2,
        pool_size=[2, 2],
        strides=2)
    # Dense Layer
    '''
    1,024 neurons
    dropout regularization rate of 0.4
    (probability of 0.4 that any given element will be dropped in training)
    '''
    pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])
    dense = tf.layers.dense(
        inputs=pool2_flat,
        units=1024,
        activation=tf.nn.relu)
    dropout = tf.layers.dropout(
        inputs=dense,
        rate=0.4,
        training=mode == tf.estimator.ModeKeys.TRAIN)
    # Dense Layer #2 (Logits Layer)
    '''
    10 neurons, one for each digit target class (0–9)
    '''
    logits = tf.layers.dense(inputs=dropout, units=10)



    predictions = {
    # Generate predictions (for PREDICT & EVAL mode)
    'classes': tf.argmax(input=logits, axis=1),
    # Add softmax_tensor to the graph, used for PREDICT
    'probabilities': tf.nn.softmax(logits, name='softmax_tensor')
    }

    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    # Calculate Loss (for TRAIN & EVAL modes)
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

    # Configure the Training Op (for Train mode)
    if(mode == tf.estimator.ModeKeys.TRAIN):
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {
    'accuracy': tf.metrics.accuracy(
        labels=labels,
        predictions=predictions['classes'])
    }
    return tf.estimator.EstimatorSpec(
        mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [4]:
def main(unused_argv):
    # Load training and eval data
    mnist = tf.contrib.learn.datasets.load_dataset('mnist')
    train_data = mnist.train.images # Returns np.array
    train_labels = np.asarray(mnist.train.labels, dtype=np.int32)
    eval_data = mnist.test.images # Returns np.array
    eval_labels = np.asarray(mnist.test.labels, dtype=np.int32)

    # Create the Estimator
    mnist_classifier = tf.estimator.Estimator(
        model_fn=cnn_model_fn, model_dir='/tmp/mnist_convnet_model')

    # Set up logging for predictions
    tensors_to_log = {'probabilities': 'softmax_tensor'}
    logging_hook = tf.train.LoggingTensorHook(
        tensors=tensors_to_log, every_n_iter=50)

    # Train the model
    train_input_fn = tf.estimator.inputs.numpy_input_fn(
        x={'x': train_data},
        y=train_labels,
        batch_size=100,
        num_epochs=None,
        shuffle=True)
    mnist_classifier.train(
        input_fn=train_input_fn,
        steps=20000,
        hooks=[logging_hook])

    # Evaluate the model and print results
    eval_input_fn = tf.estimator.inputs.numpy_input_fn(
        x={'x': eval_data},
        y=eval_labels,
        num_epochs=1,
        shuffle=False)
    eval_results = mnist_classifier.evaluate(input_fn=eval_input_fn)
    print(eval_results)

In [ ]:
if __name__ == '__main__':
  tf.app.run()

C:\Users\aamir\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MNIST-data\train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST-data\train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST-data\t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST-data\t10k-labels-idx1-ubyte.gz
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/mnist_convnet_model', '_tf_random_seed': 1, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from /tmp/mnist_convnet_model\model.ckpt-1
INFO:tensorflow:Saving checkpoints for 2 into /tmp/mnist_convnet_model\model.ckpt.
INFO:tensorflow:loss = 2

INFO:tensorflow:probabilities = [[0.09120391 0.11469881 0.07656282 0.11081631 0.10331343 0.08644284
  0.09722807 0.10629772 0.09427936 0.11915672]
 [0.07365142 0.10709445 0.07961761 0.09383025 0.11259896 0.10489896
  0.10570037 0.08717549 0.1083008  0.1271317 ]
 [0.08691432 0.11271091 0.09608115 0.10553173 0.09904259 0.09143735
  0.1037268  0.08279382 0.09782013 0.1239412 ]
 [0.09360409 0.10727247 0.08644647 0.10590837 0.10105809 0.08647792
  0.11474774 0.09623927 0.09569108 0.11255447]
 [0.10029326 0.09947181 0.08903735 0.11484531 0.11454269 0.09017167
  0.10761279 0.07772285 0.08891886 0.11738329]
 [0.0985144  0.11665127 0.08848978 0.08977677 0.10470387 0.08672033
  0.10322885 0.11043012 0.09680784 0.1046767 ]
 [0.10243918 0.10034638 0.10175865 0.1099581  0.09510116 0.08559125
  0.10432567 0.10087722 0.09247658 0.10712568]
 [0.09728289 0.10020013 0.08359592 0.10270684 0.1026415  0.09376938
  0.11287504 0.09867214 0.10261028 0.10564578]
 [0.10057502 0.12939124 0.09993751 0.07977379 0.

INFO:tensorflow:global_step/sec: 2.08496
INFO:tensorflow:loss = 2.2925372, step = 102 (47.965 sec)
INFO:tensorflow:probabilities = [[0.1126665  0.10148121 0.08998664 0.10810716 0.09224168 0.10112495
  0.10469645 0.08938594 0.08857188 0.11173759]
 [0.09270245 0.10741521 0.08536027 0.10946091 0.10253328 0.08401985
  0.10823485 0.10570935 0.09321445 0.11134936]
 [0.10242061 0.09315868 0.09345677 0.11143003 0.09631286 0.09166402
  0.1001178  0.09908885 0.09958519 0.11276529]
 [0.09981464 0.11589471 0.09405146 0.100453   0.09955578 0.08578683
  0.10453892 0.09453876 0.09919214 0.10617377]
 [0.10280867 0.11006594 0.09193227 0.0969675  0.09884337 0.09853363
  0.09559049 0.10104559 0.08280994 0.12140269]
 [0.09594677 0.11193773 0.09318595 0.11901411 0.09827878 0.09398956
  0.0996452  0.09301739 0.09306528 0.10191925]
 [0.08614064 0.10206896 0.09652636 0.09711472 0.10639535 0.10099304
  0.10550869 0.09226128 0.10104568 0.11194535]
 [0.09581379 0.10316805 0.09196778 0.11000317 0.0964862  0.08911

INFO:tensorflow:probabilities = [[0.09634613 0.10206116 0.09520032 0.0952324  0.09625349 0.09692278
  0.11959199 0.07173626 0.11320507 0.11345033]
 [0.08420173 0.11203319 0.08875601 0.11283004 0.10331574 0.08680472
  0.09791785 0.10106856 0.09268636 0.12038589]
 [0.0978754  0.10486811 0.105589   0.08408692 0.10946855 0.09812069
  0.10079031 0.09377483 0.10133517 0.10409117]
 [0.09195371 0.10548336 0.08957762 0.11490688 0.09734417 0.08792916
  0.10082247 0.09751906 0.09524528 0.11921836]
 [0.0898957  0.09564117 0.08719249 0.10802626 0.10305378 0.0874376
  0.11823037 0.09143039 0.10499592 0.11409637]
 [0.08876933 0.10440323 0.08836505 0.11064986 0.09299177 0.08866308
  0.09366485 0.09843604 0.11492055 0.11913629]
 [0.09365972 0.09797625 0.09938458 0.10287701 0.10621639 0.1003402
  0.11134068 0.08967924 0.10308895 0.09543709]
 [0.0858447  0.10956976 0.08708115 0.11485995 0.09947411 0.0913668
  0.09863    0.10603728 0.11017792 0.09695837]
 [0.103444   0.10347069 0.08652475 0.09562421 0.103